# Big Data and Data Analytics for Managers 

## Merlin John - 015027

AIM- Perform Text Classification on Coronavirus tweets using Py spark


Columns:

1) Location
2) Tweet At
3) Original Tweet
4) Label

In [ ]:
#!pip install pyspark

In [ ]:
#import os       #importing os to set environment variable
#def install_java():
#  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
#  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
#  !java -version       #check java version
#install_java()

In [1]:
#from pyspark.sql import SparkSession
from pyspark.sql.functions import length,  regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer,HashingTF
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline

In [ ]:
#spark = SparkSession.builder \
#        .config("spark.driver.memory","6G")\
#        .config("spark.driver.maxResultSize", "4G") \
#        .getOrCreate()

In [ ]:
#from google.colab import drive
#drive.mount('/gdrive')

In [ ]:
#path = "/gdrive/MyDrive/Colab_data_files/corona_nlp/Corona_NLP_test.csv"
#data=spark.read.csv(path, header = True, inferSchema=True)

In [ ]:
! hdfs dfs -mkdir -p /user/ashok/data_files/corona
! hdfs dfs -put /home/ashok/Documents/corona_nlp_test.csv  /user/ashok/data_files/corona
! hdfs dfs -ls /user/ashok/data_files/corona

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [3]:
path = "/user/ashok/data_files/corona/corona_nlp_test.csv"
data=spark.read.csv(path, header = True, inferSchema=True)

In [4]:
data.show()
data.printSchema()

+--------------------+--------------------+--------------------+----------+--------------------+------------------+
|            UserName|          ScreenName|            Location|   TweetAt|       OriginalTweet|         Sentiment|
+--------------------+--------------------+--------------------+----------+--------------------+------------------+
|                   1|               44953|                 NYC|02-03-2020|TRENDING: New Yor...|Extremely Negative|
|                   2|               44954|         Seattle, WA|02-03-2020|When I couldn't f...|          Positive|
|                   3|               44955|                null|02-03-2020|Find out how you ...|Extremely Positive|
|                   4|               44956|         Chicagoland|02-03-2020|#Panic buying hit...|              null|
|https://t.co/IASi...|                null|                null|      null|                null|              null|
|#QAnon #QAnon2018...|                null|                null|      nu

In [ ]:
data.describe().show()

# Data Prepration

In [ ]:
result_df = data.groupBy("Location").count().sort("Location", ascending=False)
result_df.show(10)

In [5]:
data=data.withColumn('length', length(data['OriginalTweet']))
data.show(10)

+--------------------+----------+-------------------+----------+--------------------+------------------+------+
|            UserName|ScreenName|           Location|   TweetAt|       OriginalTweet|         Sentiment|length|
+--------------------+----------+-------------------+----------+--------------------+------------------+------+
|                   1|     44953|                NYC|02-03-2020|TRENDING: New Yor...|Extremely Negative|   228|
|                   2|     44954|        Seattle, WA|02-03-2020|When I couldn't f...|          Positive|   193|
|                   3|     44955|               null|02-03-2020|Find out how you ...|Extremely Positive|    73|
|                   4|     44956|        Chicagoland|02-03-2020|#Panic buying hit...|              null|   209|
|https://t.co/IASi...|      null|               null|      null|                null|              null|  null|
|#QAnon #QAnon2018...|      null|               null|      null|                null|              null|

In [6]:
data=data.drop("UserName","ScreenName","Location","TweetAt")
data.filter(data["Sentiment"]=="Negative").show(10)

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|#CoronaVirus #COV...| Negative|   284|
|When your usual g...| Negative|   201|
|While you are sto...| Negative|   250|
|Two important pho...| Negative|   271|
|Fellow Uni instru...| Negative|   274|
|1/25 Part 1 - Why...| Negative|   300|
|For those self in...| Negative|   166|
|Report any and al...| Negative|   214|
|Met with members ...| Negative|   303|
|Every shop, chemi...| Negative|   270|
+--------------------+---------+------+
only showing top 10 rows



In [7]:
data = data.na.drop()
data.show()

+--------------------+--------------------+------+
|       OriginalTweet|           Sentiment|length|
+--------------------+--------------------+------+
|TRENDING: New Yor...|  Extremely Negative|   228|
|When I couldn't f...|            Positive|   193|
|Find out how you ...|  Extremely Positive|    73|
|#toiletpaper #dun...|             Neutral|   252|
|Do you remember t...|             Neutral|   205|
|Voting in the age...|            Positive|    90|
|HI TWITTER! I am ...|  Extremely Negative|   280|
|Anyone been in a ...|  Extremely Positive|   238|
|While we were bus...|            Positive|   197|
|When youre stock...|             Neutral|   187|
|#CoronaVirus #COV...|            Negative|   284|
|"For those of you...|  Extremely Positive|   279|
|Control over stoc...|  Extremely Positive|   237|
|When your usual g...|            Negative|   201|
|While you are sto...|            Negative|   250|
|"#Coronavirus is ...|"" says NELP Gove...|    72|
|Two important pho...|         

In [8]:
data = data.filter("Sentiment == 'Neutral' OR Sentiment == 'Negative' OR Sentiment == 'Positive' OR Sentiment == 'Extremely Negative' OR Sentiment == 'Extremely Positive'")
data.show()

+--------------------+------------------+------+
|       OriginalTweet|         Sentiment|length|
+--------------------+------------------+------+
|TRENDING: New Yor...|Extremely Negative|   228|
|When I couldn't f...|          Positive|   193|
|Find out how you ...|Extremely Positive|    73|
|#toiletpaper #dun...|           Neutral|   252|
|Do you remember t...|           Neutral|   205|
|Voting in the age...|          Positive|    90|
|HI TWITTER! I am ...|Extremely Negative|   280|
|Anyone been in a ...|Extremely Positive|   238|
|While we were bus...|          Positive|   197|
|When youre stock...|           Neutral|   187|
|#CoronaVirus #COV...|          Negative|   284|
|"For those of you...|Extremely Positive|   279|
|Control over stoc...|Extremely Positive|   237|
|When your usual g...|          Negative|   201|
|While you are sto...|          Negative|   250|
|Two important pho...|          Negative|   271|
|IMHO Russia's rug...|Extremely Negative|   280|
|So not due to #Br..

In [9]:
data = data.withColumn('Sentiment', regexp_replace('Sentiment', 'Extremely Negative', 'Negative'))
data = data.withColumn('Sentiment', regexp_replace('Sentiment', 'Extremely Positive', 'Positive'))

In [10]:
data.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|TRENDING: New Yor...| Negative|   228|
|When I couldn't f...| Positive|   193|
|Find out how you ...| Positive|    73|
|#toiletpaper #dun...|  Neutral|   252|
|Do you remember t...|  Neutral|   205|
|Voting in the age...| Positive|    90|
|HI TWITTER! I am ...| Negative|   280|
|Anyone been in a ...| Positive|   238|
|While we were bus...| Positive|   197|
|When youre stock...|  Neutral|   187|
|#CoronaVirus #COV...| Negative|   284|
|"For those of you...| Positive|   279|
|Control over stoc...| Positive|   237|
|When your usual g...| Negative|   201|
|While you are sto...| Negative|   250|
|Two important pho...| Negative|   271|
|IMHO Russia's rug...| Negative|   280|
|So not due to #Br...|  Neutral|   148|
|Fellow Uni instru...| Negative|   274|
|1/25 Part 1 - Why...| Negative|   300|
+--------------------+---------+------+
only showing top 20 rows



# Features Transformation

In [11]:
tokenizer=Tokenizer(inputCol="OriginalTweet", outputCol="token_text")
stopremove=StopWordsRemover(inputCol="token_text", outputCol="stop_tokens")
count_vec=CountVectorizer(inputCol="stop_tokens", outputCol="c_vec")
#hashingTF = HashingTF(inputCol="stop_tokens", outputCol="c_vec", numFeatures=5000)
idf=IDF(inputCol="c_vec", outputCol="tf_idf")
# we also need to convert our labels in numbers
ham_samp_to_num = StringIndexer(inputCol="Sentiment", outputCol='label')
clean_up = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')

# Model

In [12]:
dtc=DecisionTreeClassifier(maxDepth=15)
rf=RandomForestClassifier(numTrees=200)
nb=NaiveBayes()

# Pipeline

In [13]:
data_prep_pipeline= Pipeline(stages=[ham_samp_to_num, tokenizer, stopremove,count_vec, idf,clean_up, nb])

In [14]:
data.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|TRENDING: New Yor...| Negative|   228|
|When I couldn't f...| Positive|   193|
|Find out how you ...| Positive|    73|
|#toiletpaper #dun...|  Neutral|   252|
|Do you remember t...|  Neutral|   205|
|Voting in the age...| Positive|    90|
|HI TWITTER! I am ...| Negative|   280|
|Anyone been in a ...| Positive|   238|
|While we were bus...| Positive|   197|
|When youre stock...|  Neutral|   187|
|#CoronaVirus #COV...| Negative|   284|
|"For those of you...| Positive|   279|
|Control over stoc...| Positive|   237|
|When your usual g...| Negative|   201|
|While you are sto...| Negative|   250|
|Two important pho...| Negative|   271|
|IMHO Russia's rug...| Negative|   280|
|So not due to #Br...|  Neutral|   148|
|Fellow Uni instru...| Negative|   274|
|1/25 Part 1 - Why...| Negative|   300|
+--------------------+---------+------+
only showing top 20 rows



In [15]:
(training, testing)= data.randomSplit([0.8,0.2])

In [16]:
training.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
|  We ve got sever...| Negative|   235|
| Amazon Needs to ...| Negative|   203|
| Food poverty doe...| Negative|   225|
| This is the best...| Positive|   181|
| and major tech c...| Positive|    49|
| and vegetables&g...| Positive|    30|
| as bad as that is."| Negative|    20|
| we shouldn't pan...| Negative|    21|
|!!!!!!! French Pr...| Negative|   133|
|"""Current estima...|  Neutral|   138|
|"#Airbnb some hos...| Negative|   287|
|"#BREAKING: Ontar...| Negative|   250|
|"#Covid_19 #Hamst...| Positive|   207|
|"?? Starbucks may...| Positive|   127|
|"@AskTarget ""cor...| Positive|   244|
|"@Ryanair is at i...| Positive|   259|
|"@colemank83 offe...| Positive|   221|
|"@iamshafaatali I...| Negative|   259|
|"@phineashead @Bo...| Negative|   259|
|"@yogagirlmab Ano...| Negative|   261|
+--------------------+---------+------+
only showing top 20 rows



In [17]:
testing.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
| There is enough ...|  Neutral|    97|
|"2 Weeks #Lockdow...|  Neutral|   307|
|"@quietplacemovie...| Positive|   245|
|"A ""consumer rem...| Positive|   259|
|"Be like my girlf...| Positive|   277|
|"My grocery store...| Negative|   279|
|"Yesterday in my ...| Negative|   267|
|#BBCqt @MattHanco...|  Neutral|    70|
|#COVID2019 whiske...|  Neutral|    72|
|#Coronavirus is h...| Positive|   162|
|#Coronavirus my n...| Negative|   263|
|#Covid_19 #Corona...| Negative|   277|
|#Covid_19 #corona...| Positive|   292|
|#Covid_19 is pand...| Positive|   210|
|#Covid_19 or not,...| Positive|   132|
|#ITV #Vera openin...|  Neutral|   101|
|#News cycles rela...|  Neutral|    99|
|#Oil vs #coronavi...| Negative|   267|
|#Quarantine #coro...| Positive|   272|
|#Unorthodox measu...| Positive|   208|
+--------------------+---------+------+
only showing top 20 rows



In [18]:
%%time
model = data_prep_pipeline.fit(training)

CPU times: user 49.3 ms, sys: 12.9 ms, total: 62.2 ms
Wall time: 8.4 s


In [19]:
model

PipelineModel_85e191c53380

### Decision Tree Classifier

In [20]:
dx = spark.createDataFrame(testing.take(2))

In [21]:
dx.show()

+--------------------+---------+------+
|       OriginalTweet|Sentiment|length|
+--------------------+---------+------+
| There is enough ...|  Neutral|    97|
|"2 Weeks #Lockdow...|  Neutral|   307|
+--------------------+---------+------+



In [22]:
test_results=model.transform(testing)

In [23]:
test_results.show()

+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|       OriginalTweet|Sentiment|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|       rawPrediction|         probability|prediction|
+--------------------+---------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| There is enough ...|  Neutral|    97|  2.0|[, there, is, eno...|[, enough, everyo...|(11863,[3,34,40,8...|(11863,[3,34,40,8...|(11864,[3,34,40,8...|[-325.04929382814...|[0.90302100509302...|       0.0|
|"2 Weeks #Lockdow...|  Neutral|   307|  2.0|["2, weeks, #lock...|["2, weeks, #lock...|(11863,[1,10,26,3...|(11863,[1,10,26,3...|(11864,[1,10,26,3...|[-688.79452539732...|[1.5137851922

In [24]:
acc_eval=MulticlassClassificationEvaluator()
acc=acc_eval.evaluate(test_results)

In [25]:
print ("Accuracy of the model is::", acc)

Accuracy of the model is:: 0.5281473038256354


In [ ]:
###############